# imports:

In [ ]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *
from PredictionPipeline.PredictionFunctions import *

import os
from Utils import cacheOnDisk
from queue.qp import qp,fakeqp
from addloglevels import sethandlers

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression
import shap
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,SelectKBest,chi2,mutual_info_classif,f_classif
from sklearn.naive_bayes import GaussianNB

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

# prediction function:

In [ ]:
def predictBinaryByDistMat(Y,YName,X,XName,ResultFolder,model,modelName,model_params,n_splits,useCV=True,stratifiedCV=True):
    
    predResultsDF=pd.DataFrame()
    #(1) arrange result folders:
    print 'arranging  result folders:'
    
    predResultsDFFolder='%s/%s/%s_predictions/predictionDFs' %(ResultFolder,modelName,YName) #define folder for all result dfs in this model
    if not isdir(predResultsDFFolder):
        makedirs(predResultsDFFolder)
        print 'generating predResultsDFFolder %s' %predResultsDFFolder      
    predResultsfigFolder='%s/%s/%s_predictions/figs' %(ResultFolder,modelName,YName) #define folder for figs in this model
    if not isdir(predResultsfigFolder):
        makedirs(predResultsfigFolder)
        print 'predResultsfigFolder %s' %predResultsfigFolder

    #generate result DF name based on model_params:
    d=OrderedDict(sorted(model_params.items(), key=lambda t: t[0]))
    try:
        d2=d.copy()
        del d2['numthreads']
    except:
        d2=d.copy()
    predResultsDFName='_'.join(['%s%s' %(key.replace('_',''), value) for (key, value) in d2.items()]) #generate a file name based on params
    predResultsDFName=predResultsDFName.replace('.','-')
    if model_params=={}:
        predResultsDFName='defaultParams'
    if useCV:
        predResultsDFName='%s_CV%s' %(predResultsDFName,n_splits)
    predResultsDFName='%s_%s' %(XName,predResultsDFName)
    
    predResultsDFfile='%s/%s.xlsx' %(predResultsDFFolder,predResultsDFName)
    existingDFs=[f for f in listdir( predResultsDFFolder) if isfile(join(predResultsDFFolder, f))]
    

    if predResultsDFName not in existingDFs:  
        # common processing of X and y:
        # leave only common samples in each df (X and seq1data)
        X=X.loc[[str(x) for x in X.index.tolist() if x in Y.index],:]
        X=X.sort_index()
        print 'X shape is %s_%s' %(X.shape[0], X.shape[1])
        print 'the 100th sample in X is %s' %X.index[100]
        print X.iloc[:3,:3]
        Y=Y.loc[[str(x) for x in Y.index.tolist() if x in X.index]]
        Y=Y.sort_index()
#         oldColName=str(Y.columns.values[0])
#         Y=Y.rename(columns={oldColName:'Class'})
        print 'Y shape is %s' %(Y.shape[0])
        print 'the 100th sample in Y is %s' %Y.index[100]
        print Y.head(3)

        #(2) model fitting and predictions:
        if useCV:
            print 'splitting train_test using cross validation with %s splits...' %n_splits
            
            if stratifiedCV:
                group_kfold = StratifiedKFold(n_splits=n_splits)
                groups=None
            else:
                group_kfold = GroupKFold(n_splits=n_splits)
                groups = np.array(range(X.shape[0]))
                
        
            y_pred_df = pd.DataFrame(index=Y.index, columns=['pred_proba'])
            i=0
            for train_index, val_index in group_kfold.split(X, Y,groups):
                print i
                i+=1
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = Y.loc[X_train.index], Y.loc[X_val.index]
                
                print 'fraction of 1s in train set=%s' %(float(y_train.sum())/len(y_train))
                print 'fraction of 1s in test set=%s' %(float(y_val.sum())/len(y_val))

                # creating the model object
                m = model(**model_params)

                # fitting the training
                if modelName=='LGBMClassifier':
                    m.fit(X_train, y_train,
                            eval_set=[(X_val, y_val)],
                            early_stopping_rounds=None, verbose=-1)
                else:
                    m.fit(X_train, y_train)
                # getting the predictions for the test
                y_pred_proba = m.predict_proba(X_val)
                y_pred_df.loc[y_val.index, :] = np.expand_dims(y_pred_proba[:,1], 1)
        else:
            test_size = 0.33
            print 'using normal train_test split with test_size=%s' %test_size
            from sklearn.model_selection import train_test_split  
            # seed = 7
            
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, stratify=Y)
            print 'fitting model...'
            model = model
            model.fit(X_train, y_train)
            # make predictions for test data
            print 'predicting...'
            y_pred = model.predict(X_test)
            predictions = [round(value) for value in y_pred]
            # evaluate predictions
            from sklearn.metrics import accuracy_score
            accuracy = accuracy_score(y_test, y_pred)
            print("Accuracy: %.2f%%" % (accuracy * 100.0))
            y_pred_df = pd.DataFrame(index=Y.index, data={'pred':y_pred})

        #(4) plots:
        print 'generating plots...'

        # this plot shows the probabilities returned by the predictor colored by the class
        plt.figure(figsize=(3,2))
        plt.scatter(range(y_pred_df.shape[0]), y_pred_df.pred_proba, c=Y)
        plt.show()

        # plot ROC and PR curves

        from sklearn import metrics
        fpr, tpr, thresholds = metrics.roc_curve(Y+1, y_pred_df.pred_proba, pos_label=2)
        roc_auc = metrics.auc(fpr, tpr)
        print roc_auc

        plt.figure(figsize=(20,7))
        plt.subplot(1,2,1)
        lw = 2
        plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.3f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate', fontsize=15)
        plt.ylabel('True Positive Rate', fontsize=15)
        plt.title('ROC curve - ' , fontsize=20)
        plt.legend(loc="lower right", fontsize=15)

        plt.subplot(1,2,2)
        precision, recall, _ = metrics.precision_recall_curve(Y, y_pred_df.pred_proba)
        pr_auc = metrics.auc(recall, precision)

        plt.step(recall, precision, color='b', alpha=0.2, where='post')
        plt.fill_between(recall, precision, step='post', alpha=0.5,
                         color='darkorange', label='Precision Recall curve - AUC = {0:0.3f}'.format(pr_auc))
        plt.plot([0, 1], [Y.sum()/Y.shape[0], Y.sum()/Y.shape[0]], color='navy', lw=lw, linestyle='--')
        plt.xlabel('Recall', fontsize=15)
        plt.ylabel('Precision', fontsize=15)
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.title('Precision Recall curve', fontsize=20)
        plt.legend(loc="upper right", fontsize=15)

        if roc_auc>0.55:
            predResultsFigfile='%s/%s_ROC-PR.png' %(predResultsfigFolder,predResultsDFName)
            plt.savefig(predResultsFigfile, bbox_inches='tight')

        plt.show()

        #(5) generate summarizing df:
        print 'generating summarizing df'
        predResultsDF.loc[0,'Yname']=YName
        predResultsDF.loc[0,'Xname']=XName
        predResultsDF.loc[0,'modelName']=modelName
        predResultsDF.loc[0,'roc_auc']=round(roc_auc,3)
        predResultsDF.loc[0,'pr_auc']=round(pr_auc,3)
        predResultsDF.loc[0,'perc_pos_target']=round(float(Y.sum())/len(Y),3)
        predResultsDF.loc[0,'pr_auc_corrected']=round(pr_auc,3)-round(float(Y.sum())/len(Y),3)
        predResultsDF.loc[0,'useCV']=useCV
        if useCV:
            predResultsDF.loc[0,'n_splits']=n_splits
        
        
        for (key, value) in OrderedDict(sorted(model_params.items(), key=lambda t: t[0])).items():
            predResultsDF.loc[0,key]=value
        
        nPos=int(Y.sum())
        nNeg=len(Y)-nPos
        
        print nPos, nNeg

        predResultsDF.loc[0,'nPos']=nPos
        predResultsDF.loc[0,'nNeg']=nNeg
        
        
        predResultsDF.to_excel(predResultsDFfile)
        print predResultsDFfile
        
    else:
        print 'this prediction already exists'
    
    return predResultsDF
    
    
    
    
    
    
    

# Prediction pipeline function
calls the prediction function

In [ ]:
def predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList):
    
    #(1) get phenotypeDF:
    if usePhenotype:
        if genPhenotypeDF:
            ## generate phenotype df if necessary
            print 'generating phenotypeDF...'
            ### get the original phenotype file(s) - concat if necessary:
            
            PNPphenotypeFile='%s/TCR_real_data/NewPhenotypicData/PNP530_AgeGenderBMIcreatSmoking.xlsx' %MyPath
            CardioPhenotypeFile='%s/TCR_real_data/CardioSamples/phenotypicData/Cardio126phen.xlsx' %MyPath

            PNPphenotypeDF=pd.read_excel(PNPphenotypeFile).set_index('BD')
            CardiophenotypeDF=pd.read_excel(CardioPhenotypeFile).set_index('BD')

            if datasetType=='PNP':
                phenotypeDF=PNPphenotypeDF
            elif datasetType=='PNP_Cardio':
                phenotypeDF=pd.concat([PNPphenotypeDF,CardiophenotypeDF])
                phenotypeDF=phenotypeDF.drop(['RegistrationCode','DM','Diagnosis'],axis=1)

#             phenotypeDF['isCardio']=np.where(phenotypeDF.index.str.replace('BD','').astype(int)>949,1,0)
            print 'phenotypeDF shape is %s_%s' %(phenotypeDF.shape[0],phenotypeDF.shape[1])
            print phenotypeDF.tail()
            
            ### generate dummy variables:
            #the function gen_dummies was copied to Feature_phenotype_functions.py
            toDummyColList=['Gender','Smoking','PCRplate']
            df=phenotypeDF.copy()
            phenotypeDF=gen_dummies(df,toDummyColList)
            print 'pehonotypeDF head after dummy addition:'
            phenotypeDF.head()
            
            #save phenotypeDF:
            if datasetType=='PNP':
                f2='%s/TCR_real_data/NewPhenotypicData/PNP530_AgeGenderBMIcreatSmoking_withDummies.xlsx' %MyPath
            elif datasetType=='PNP_Cardio':
                f2='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
            phenotypeDF.to_excel(f2)
        else:
            print 'loading phenotype file...'
            phenotypeDF=pd.read_excel(phenotypefile).set_index('BD')
            print phenotypeDF.shape
            phenotypeDF.tail()
            
        #
        # Xtypes:
        allNum=['Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Gender_Male', 'Smoking_Past', 'Smoking_Yes',
               'PCR_Plate1', 'PCR_Plate10', 'PCR_Plate2', 'PCR_Plate3',
               'PCR_Plate4', 'PCR_Plate5', 'PCR_Plate6', 'PCR_Plate7',
               'PCR_Plate8', 'PCR_Plate9']
        small=[ 'Age', 'BMI', 'Creatinine', 'isCardio', 'Gender_Male','Smoking_Past', 'Smoking_Yes']
        smallNoCardio=['Age', 'BMI', 'Creatinine', 'Gender_Male','Smoking_Past', 'Smoking_Yes']
        allNumNoPlate=['Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Gender_Male', 'Smoking_Past', 'Smoking_Yes']

        if phenotypeDFname=='small':
            Xcols=small
        elif phenotypeDFname=='allNum':
            Xcols=allNum
        elif phenotypeDFname=='smallNoCardio':
            Xcols=smallNoCardio
        elif phenotypeDFname=='allNumNoPlate':
            Xcols=allNumNoPlate
            
        phenotypeDF=phenotypeDF[Xcols]
        print 'final phenotype shape is %s_%s' %(phenotypeDF.shape[0],phenotypeDF.shape[1])

        
        #(2) get TCRdf:
        if useTCRdf or usePCAdf:
            ## ** load shared sequence matrix
            print 'loading shared sequence matrix:'
            TCRdfShortName=TCRdfName.replace('sharingMatrix_PNP530Cardio126MatchedSamples_','')
            TCRdfShortName=TCRdfShortName.replace('__','_')

            f1='%s/sharingAnalysis/%s' %(datasetFolder,TCRdfName)
            TCRdf=pd.read_pickle(f1)
            print 'TCRdf shape is %s_%s' %(TCRdf.shape[0],TCRdf.shape[1])
            print 'TCRdf head:'
            print TCRdf.iloc[:4,:4]
        
        #(3) get PCAdf:
        if usePCAdf:
            ## **generate PCs as features:
            if n_comp is None:
                n_comp=5
            print 'generating PCAdf with %s PCs based on TCRdf:' %n_comp
            PCAdf=PCAfunc(TCRdf,n_comp,isSparse)
            for c in ['BDindex','isCardio']:
                try:
                    PCAdf=PCAdf.drop(c,axis=1)
                except:
                    print 'PCAdf doesnt include %s column' %c
            print 'PCAdf shape is %s_%s' %(PCAdf.shape[0],PCAdf.shape[1])
            print 'PCAdf HEAD:'
            print PCAdf.head()
          
        #(4) get TCRfeature:
        if useTCRfeatures:
            if getTCRfeatures:
                print 'generating new feature DF...this will take a while and will print out many things...'
                ### calculate features - from PopulationAnalaysis_new_version.ipynb
                data_folder=datasetFolder.replace('%s/' %MyPath,'')
                dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected' %data_folder
                filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
                filenames=[f.strip('.tsv') for f in filenames]
                filenames=[f.strip('.xlsx') for f in filenames]
                print 'number of samples to extract features is %s' %len(filenames)
                for n,sample_name in enumerate(filenames): 
                    print  n,sample_name
                    gen_descriptive_stats(sample_name,data_folder,newColumnList=None)
                    gen_geneUsageCount(sample_name,data_folder,newColumnList=None)
                ### generate feature DFs - based on 'Generate features DF' notebook
                seqTypeList = ['Total', 'Prod','nonProd']
                dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/Total' %data_folder
                FeatureGroups=listdir(dfs_folder)
                for seqType in seqTypeList:
                    print seqType
                    gen_featureSummaryDF_forSeqType(seqType,data_folder,datasetName,FeatureGroups)
                    
                data_folder=datasetFolder.replace('%s/' %MyPath,'')
                gen_merged_feature_df_for_dataset(data_folder,datasetName)
            else:
                print 'loading featureDF...'
                f2='%s/featureSummaryDFs/%s_allFeatures' %(datasetFolder,datasetName)
                TCRfeatureDF=pd.read_pickle(f2)
                print 'TCRfeatureDF shape is %s_%s' %(TCRfeatureDF.shape[0],TCRfeatureDF.shape[1])
                print 'TCRfeatureDF head:'
                print TCRfeatureDF.iloc[:4,:4]
                
            ## ** fillna's in featureDF:
            #get number of columns with number of nan values?
            print 'fill nans in featureDF...:'
            print 'columns with top numbers of nan values:'
            print TCRfeatureDF.isnull().sum().sort_values(ascending=False).head()
            print 'value counts of nan numbers per column:'
            print TCRfeatureDF.isnull().sum().value_counts(dropna=False).sort_values(ascending=False).head()
            # find how many columns include nans:
            print 'number of columns with nan values = %s' %len(TCRfeatureDF.loc[:,TCRfeatureDF.isnull().sum()>0].columns.tolist())
            print 'total number of columns is %s' %len(TCRfeatureDF.columns)
            #get list of columns that inlcude nans and should be fillna-ed with 0s:
            columnsToFill=TCRfeatureDF.loc[:,TCRfeatureDF.isnull().sum()>0].columns.tolist()
            columnsToFill_clean=[x for x in columnsToFill if 'norm' not in x]
            print 'number of columns that include nans and their name doesnt include -norm- is %s' %len(columnsToFill_clean)
            print 'these columns will be fillna-ed with 0s'

            #fillna with zeros
            TCRfeatureDF[columnsToFill_clean]=TCRfeatureDF[columnsToFill_clean].fillna(0)

            #check how many columns remained with nans:
            print 'now the number of columns with nan values is %s' %len(TCRfeatureDF.loc[:,TCRfeatureDF.isnull().sum()>0].columns.tolist())

        #(5) get y:
        y=targetDF[targetName]
        print 'y shape is %s' %y.shape
        print 'y head:'
        print y.head()
        print y.tail()
        
        
        #(6) edit X- filter with sampleList, filter nan columns, fillna, filter for y rows
        
        #build X from its components:
        print 'building X from its components:'
        XcomponentList=[]
        if usePhenotype:
            XcomponentList.append(phenotypeDF)
        if useTCRdf:
            XcomponentList.append(TCRdf)
        if useTCRfeatures:
            XcomponentList.append(TCRfeatureDF)
        if usePCAdf:
            XcomponentList.append(PCAdf)
        print 'number of Xcomponents to be used is %s' %len(XcomponentList)
        
        #edit sample names and merge all X components:
        for n,comp in enumerate(XcomponentList):
#             print n
#             print 'comp shape is %s_%s' %(comp.shape[0],comp.shape[1])
#             print 'editing sample names..'
            comp=editSampleNames(comp)
            if n==0:
                X=comp
            else:
                X=pd.merge(X,comp, how='inner', right_index=True,left_index=True)
        print 'combined X shape is %s_%s' %(X.shape[0],X.shape[1])
        print len(X.index.tolist())
        
        #if model is 'logit', drop columns that contains nans, except for phenotype columns; fillna in phenotype columns
        #and check for rows with nan values:

        if ('log' in modelName) or ('Log' in modelName):
            print 'model is logit, dropping nan feature columns and filling nas in phenotype columns with median column values...'
            nullSum=X.isnull().sum()
            ignoreColumns=nullSum[nullSum>0].index.tolist()
            for column in phenotypeDF.columns.values:
                if column in ignoreColumns:
                    ignoreColumns.remove(column)
            X=X.drop(ignoreColumns,axis=1)
            print 'X shape after dropping nan columns is %s_%s' %(X.shape[0],X.shape[1])
            for column in phenotypeDF.columns.values:
                X[column]= X[column].fillna(X[column].median())
            X=X.dropna(how='any')
            print 'Xshape after dropping rows with nans=%s_%s' %(X.shape[0],X.shape[1])

        # drop samples that are not in y
        print 'dropping X rows that do not appear in y:'
        ysamples=y.index.tolist()
        X=X.loc[ysamples,:]
        X=X.dropna(how='all')
        print 'Xshape after dropping rows that do not appear in y = %s_%s:' %(X.shape[0],X.shape[1])
        
        #filter X rows by sampleList:
        if sampleListName is not None:
            print 'filtering X rows with sample List = %s' %sampleListName
            X=X.loc[sampleList,:]
            print 'Xshape after filtering rows with sample list = %s_%s:' %(X.shape[0],X.shape[1])
        
        
        #(7) edit y
        # leave only rows that appear in X
        print 'y shape before filtering for X sample is %s' %y.shape[0]
        Xsamples=X.index.tolist()
        y=y.loc[Xsamples]
        # y=y.dropna(how='all')
        print 'y shape after filtering for X sample is %s' %y.shape[0]
        
        #(8) feature selection: placeholder
        
        #(9) model prediction: yield df, plots
        print 'calculating model prediction'
        Y=y
        predResultsDF=predictBinaryByDistMat(Y,YName,X,XName,ResultFolder,model,modelName,model_params,n_splits,
                                             useCV,stratifiedCV)
        print predResultsDF
        print 'done!!'
    
        
        
        
        


#  define input and call prediction pipeline:

## run for PNP530Cardio126Matchedss9000_onlyPhenotypes

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= lgb.LGBMClassifier
modelName='LGBMClassifier'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_onlyPhenotypes'
## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
## PCA- indicate n_comp and is Sparse, indicate feature selection
usePhenotype=True #True/False
useTCRdf=False #True/False
useTCRfeatures=False #True/False
usePCAdf=False #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List 

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList)


## run for PNP530Cardio126Matchedss9000 - Phenotypes+TCR

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= lgb.LGBMClassifier
modelName='LGBMClassifier'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_Phenotypes&TCRdf' #### always change!!!!
usePhenotype=True #True/False
useTCRdf=True #True/False
useTCRfeatures=False #True/False
usePCAdf=False #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List 

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList)


## run for PNP530Cardio126Matchedss9000 - Phenotypes+PCA

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= lgb.LGBMClassifier
modelName='LGBMClassifier'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_Phenotypes&PCA' #### always change!!!!
usePhenotype=True #True/False
useTCRdf=False#True/False
useTCRfeatures=False #True/False
usePCAdf=True #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List 

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList)


## run for PNP530Cardio126Matchedss9000 - Phenotypes+features

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= lgb.LGBMClassifier
modelName='LGBMClassifier'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_Phenotypes&TCRfeatures' #### always change!!!!
usePhenotype=True #True/False
useTCRdf=False #True/False
useTCRfeatures=True #True/False
usePCAdf=False #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List 

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList)


## run for PNP530Cardio126Matchedss9000 - Phenotypes only LOGIT!

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= LogisticRegression
modelName='LogisticRegression'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
# model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}
model_params={}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_PhenotypesOnly' #### always change!!!!
usePhenotype=True #True/False
useTCRdf=False #True/False
useTCRfeatures=False #True/False
usePCAdf=False #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List 

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList)


## run for PNP530Cardio126Matchedss9000 - Phenotypes+TCRdf - LOGIT!

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= LogisticRegression
modelName='LogisticRegression'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
# model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}
model_params={}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_Phenotypes&TCRdf' #### always change!!!!
usePhenotype=True #True/False
useTCRdf=True #True/False
useTCRfeatures=False #True/False
usePCAdf=False #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List 

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList)


## run for PNP530Cardio126Matchedss9000 - Phenotypes+PCAdf - LOGIT!

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= LogisticRegression
modelName='LogisticRegression'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
# model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}
model_params={}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_Phenotypes&PCA' #### always change!!!!
usePhenotype=True #True/False
useTCRdf=False #True/False
useTCRfeatures=False #True/False
usePCAdf=True #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List 

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList)


## run for PNP530Cardio126Matchedss9000 - Phenotypes+TCRfeatures - LOGIT!

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= LogisticRegression
modelName='LogisticRegression'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
# model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}
model_params={}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_Phenotypes&TCRfeatures' #### always change!!!!
usePhenotype=True #True/False
useTCRdf=False #True/False
useTCRfeatures=True #True/False
usePCAdf=False #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List 

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList)


# run again with regularization:

## matched samples, phenotypes+PCA+features, no regularization, LGB

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= lgb.LGBMClassifier #LogisticRegression/lgb.LGBMClassifier
modelName='LGBMClassifier' #'LogisticRegression'/'LGBMClassifier'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}
# model_params={}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_Phenotypes&PCA&TCRfeatures' #### always change!!!!
usePhenotype=True #True/False
useTCRdf=False #True/False
useTCRfeatures=True #True/False
usePCAdf=True #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List 

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList)


## matched samples, phenotypes+PCA+features, with regularization, LGB

In [ ]:
# loop over model parameters:

n_estimators_list=[20,100,1000]
learning_rate_list=[0.01,0.05,0.1]
maxDepth_numLeaves_pair_list=[(3,4),(3,8),(6,30),(6,60)]
minDataLeaf_list=[20,40]
baggingFraction_list=[1,0.5]

count=1
for n_estimators in n_estimators_list:
    for learning_rate in learning_rate_list:
        for min_data_in_leaf in minDataLeaf_list:
            for bagging_fraction in baggingFraction_list:
                for pair in maxDepth_numLeaves_pair_list:
                    
                    print count
                    count=count+1
                    
                    # setting parameters:
                    max_depth=pair[0]
                    num_leaves=pair[1]
                    
                    bagging_freq=1
                    bagging_seed=3
                    
                    
                    # (1) general definitions:
                    datasetType='PNP_Cardio' #PNP/PNP_Cardio
                    #nSeqSS=9000 #5000/9000/15000/'all'

                    #(2) model definitions:
                    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
                    if not isdir(ResultFolder):
                        makedirs(ResultFolder)
                    model= XGBClassifier #LogisticRegression/lgb.LGBMClassifier/XGBClassifier
                    modelName='XGBClassifier' #'LogisticRegression'/'LGBMClassifier'/'XGBClassifier'
                    n_splits=3
                    useCV=True
                    stratifiedCV=True

                    model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_leaves':num_leaves,
                                  'min_data_in_leaf':min_data_in_leaf,'bagging_fraction':bagging_fraction,'bagging_freq':bagging_freq,'bagging_seed':bagging_seed,
                                  'num_threads':2}
#                     model_params={}

                    #(3) target definition:
                    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
                    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
                    targetName='isCardio'
                    YName=targetName

                    #  (4) feature composition:
                    XName='PNP530Cardio126Matchedss9000_Phenotypes&PCA&TCRfeatures' #### always change!!!!
                    usePhenotype=True #True/False
                    useTCRdf=False #True/False
                    useTCRfeatures=True #True/False
                    usePCAdf=True #True/False

                    # (5) phenotypeDF:

                    genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
                    phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
                    phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


                    # (6) TCRdf:
                    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
                    datasetName='PNP530Cardio126_ss9000rep1'
                    TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

                    #(7) PCAdf (based on the TCRdf selected)
                    n_comp=10 # int or None 
                    isSparse=True #True/False

                    #(8) TCRfeatures:
                    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

                    #(9) X and y processing:
                    folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
                    sampleListName='matched86List' ################IMPORTANT!!! ###################
                    matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
                    sampleList= matched86List 


                    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                          YName,targetDF,
                                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                          getTCRfeatures,sampleListName,sampleList)


### get results:

In [ ]:
folder='%s/TCR_real_data/Predictions/isCardio_predictions/LGBMClassifier/predictionDFs' %MyPath
results=concat_summarizing_dfs_excel(folder)
results.head()

In [ ]:
results=results.sort_values(by='roc_auc', ascending=False)

In [ ]:
f1='%s/TCR_real_data/Predictions/isCardio_predictions/LGBMClassifier/resultSummary_%s.xlsx' %(MyPath, cdate)
results.to_excel(f1)

# try sklearn's xgboost

In [ ]:
# loop over model parameters:

n_estimators_list=[20,100,1000]
learning_rate_list=[0.01,0.05,0.1]
maxDepth_numLeaves_pair_list=[(3,4),(3,8),(6,30),(6,60)]
minDataLeaf_list=[20,40]
baggingFraction_list=[1,0.5]

count=1
for n_estimators in n_estimators_list:
    for learning_rate in learning_rate_list:
        for min_data_in_leaf in minDataLeaf_list:
            for bagging_fraction in baggingFraction_list:
                for pair in maxDepth_numLeaves_pair_list:
                    
                    print count
                    count=count+1
                    
                    # setting parameters:
                    max_depth=pair[0]
                    num_leaves=pair[1]
                    
                    bagging_freq=1
                    bagging_seed=3
                    
                    
                    # (1) general definitions:
                    datasetType='PNP_Cardio' #PNP/PNP_Cardio
                    #nSeqSS=9000 #5000/9000/15000/'all'

                    #(2) model definitions:
                    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
                    if not isdir(ResultFolder):
                        makedirs(ResultFolder)
                    model= lgb.LGBMClassifier #LogisticRegression/lgb.LGBMClassifier
                    modelName='LGBMClassifier' #'LogisticRegression'/'LGBMClassifier'
                    n_splits=3
                    useCV=True
                    stratifiedCV=True

                    model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_leaves':num_leaves,
                                  'min_data_in_leaf':min_data_in_leaf,'bagging_fraction':bagging_fraction,'bagging_freq':bagging_freq,'bagging_seed':bagging_seed,
                                  'num_threads':2}
#                     model_params={}

                    #(3) target definition:
                    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
                    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
                    targetName='isCardio'
                    YName=targetName

                    #  (4) feature composition:
                    XName='PNP530Cardio126Matchedss9000_Phenotypes&PCA&TCRfeatures' #### always change!!!!
                    usePhenotype=True #True/False
                    useTCRdf=False #True/False
                    useTCRfeatures=True #True/False
                    usePCAdf=True #True/False

                    # (5) phenotypeDF:

                    genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
                    phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
                    phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


                    # (6) TCRdf:
                    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
                    datasetName='PNP530Cardio126_ss9000rep1'
                    TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

                    #(7) PCAdf (based on the TCRdf selected)
                    n_comp=10 # int or None 
                    isSparse=True #True/False

                    #(8) TCRfeatures:
                    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

                    #(9) X and y processing:
                    folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
                    sampleListName='matched86List' ################IMPORTANT!!! ###################
                    matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
                    sampleList= matched86List 


                    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                          YName,targetDF,
                                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                          getTCRfeatures,sampleListName,sampleList)



# develop feature selectio  and feature importance tools:

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= lgb.LGBMClassifier
modelName='LGBMClassifier'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=1000
max_depth=3
model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_phenotypes&PCA&TCRfeature'
## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
## PCA- indicate n_comp and is Sparse, indicate feature selection
usePhenotype=True #True/False
useTCRdf=False #True/False
useTCRfeatures=True #True/False
usePCAdf=True #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List

filterFeaturesByCorr=None #None or float between 0 and 1
featureSelectionUsingModel=False # True/False
C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
k=150 #None (will be set to 50 / int - defines number of k best))

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList,
                      filterFeaturesByCorr,featureSelectionUsingModel,C,
                      filterFeaturesByUnivariate,k)


# try using naive_bayes model:

In [ ]:
# (1) general definitions:
datasetType='PNP_Cardio' #PNP/PNP_Cardio
#nSeqSS=9000 #5000/9000/15000/'all'

#(2) model definitions:
ResultFolder='%s/TCR_real_data/Predictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= GaussianNB
modelName='GaussianNB'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=1000
max_depth=3
# model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}
model_params={}

#(3) target definition:
targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
targetName='isCardio'
YName=targetName

#  (4) feature composition:
XName='PNP530Cardio126Matchedss9000_phenotypes&PCA&TCRfeature'
## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
## PCA- indicate n_comp and is Sparse, indicate feature selection
usePhenotype=True #True/False
useTCRdf=False #True/False
useTCRfeatures=True #True/False
usePCAdf=True #True/False

# (5) phenotypeDF:

genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


# (6) TCRdf:
datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
datasetName='PNP530Cardio126_ss9000rep1'
TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

#(7) PCAdf (based on the TCRdf selected)
n_comp=10 # int or None 
isSparse=True #True/False

#(8) TCRfeatures:
getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

#(9) X and y processing:
folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
sampleListName='matched86List' ################IMPORTANT!!! ###################
matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
sampleList= matched86List

filterFeaturesByCorr=None #None or float between 0 and 1
featureSelectionUsingModel=False # True/False
C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
k=150 #None (will be set to 50 / int - defines number of k best))

   
predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                      YName,targetDF,
                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                      getTCRfeatures,sampleListName,sampleList,
                      filterFeaturesByCorr,featureSelectionUsingModel,C,
                      filterFeaturesByUnivariate,k)


# try different models and different feature selections:

## calculate phenotype only prediction using each model with default parameters:

In [ ]:
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2}),
                   (GaussianNB,'GaussianNB',{})]


for n,modelAndParams in enumerate(modelAndParams_list):
    
    model=modelAndParams[0]
    modelName=modelAndParams[1]
    model_params=modelAndParams[2]
    
    
    print n,modelName
    
    # (1) general definitions:
    datasetType='PNP_Cardio' #PNP/PNP_Cardio
    #nSeqSS=9000 #5000/9000/15000/'all'

    #(2) model definitions:
    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
    if not isdir(ResultFolder):
        makedirs(ResultFolder)

    n_splits=3
    useCV=True
    stratifiedCV=True

    #(3) target definition:
    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
    targetName='isCardio'
    YName=targetName

    #  (4) feature composition:
    XName='PNP530Cardio126Matchedss9000_phenotypesOnly'
    ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
    ## PCA- indicate n_comp and is Sparse, indicate feature selection
    usePhenotype=True #True/False
    useTCRdf=False #True/False
    useTCRfeatures=False #True/False
    usePCAdf=False #True/False

    # (5) phenotypeDF:

    genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
    phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
    phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


    # (6) TCRdf:
    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
    datasetName='PNP530Cardio126_ss9000rep1'
    TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

    #(7) PCAdf (based on the TCRdf selected)
    n_comp=10 # int or None 
    isSparse=True #True/False

    #(8) TCRfeatures:
    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

    #(9) X and y processing:
    folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
    sampleListName='matched86List' ################IMPORTANT!!! ###################
    matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
    sampleList= matched86List

    filterFeaturesByCorr=None #None or float between 0 and 1
    featureSelectionUsingModel=False # True/False
    C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
    filterFeaturesByUnivariate=None     # None/chi2/f_classif/mutual_info_classif
    k=150 #None (will be set to 50 / int - defines number of k best))


    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                          YName,targetDF,
                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                          getTCRfeatures,sampleListName,sampleList,
                          filterFeaturesByCorr,featureSelectionUsingModel,C,
                          filterFeaturesByUnivariate,k)

    

## try different models with different feature selection combinations:

In [ ]:
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2}),
                   (GaussianNB,'GaussianNB',{})]

filterFeaturesByCorr_list=[None]
featureSelectionUsingModel_list=[True,False]
filterFeaturesByUnivariate_list=[None,f_classif,mutual_info_classif]
n_comp_list=[10,100]


count=1
for modelAndParams in modelAndParams_list:
    
    model=modelAndParams[0]
    modelName=modelAndParams[1]
    model_params=modelAndParams[2]
    
    for filterFeaturesByCorr in filterFeaturesByCorr_list:
        for featureSelectionUsingModel in featureSelectionUsingModel_list:
            for filterFeaturesByUnivariate in filterFeaturesByUnivariate_list:
                for n_comp in n_comp_list:

                    print count
                    count=count+1
                    C=0.5
                    k=150

                    # (1) general definitions:
                    datasetType='PNP_Cardio' #PNP/PNP_Cardio
                    #nSeqSS=9000 #5000/9000/15000/'all'

                    #(2) model definitions:
                    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
                    if not isdir(ResultFolder):
                        makedirs(ResultFolder)
                    n_splits=3
                    useCV=True
                    stratifiedCV=True

                    #(3) target definition:
                    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
                    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
                    targetName='isCardio'
                    YName=targetName

                    #  (4) feature composition:
                    XName='PNP530Cardio126Matchedss9000_phenotypes&PCA&TCRfeature'
                    ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
                    ## PCA- indicate n_comp and is Sparse, indicate feature selection
                    usePhenotype=True #True/False
                    useTCRdf=False #True/False
                    useTCRfeatures=True #True/False
                    usePCAdf=True #True/False

                    # (5) phenotypeDF:

                    genPhenotypeDF=False #True/False. no sense for 'True'  if usePhenotype=False
                    phenotypefile='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
                    phenotypeDFname='smallNoCardio' #('small','allNum and more- see step #6.2


                    # (6) TCRdf:
                    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss9000rep1' %MyPath
                    datasetName='PNP530Cardio126_ss9000rep1'
                    TCRdfName='sharingMatrix_PNP530Cardio126MatchedSamples_minNshared2_RA_onlyProductiveTrue__percShared10__binary'

                    #(7) PCAdf (based on the TCRdf selected)
                    isSparse=True #True/False

                    #(8) TCRfeatures:
                    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

                    #(9) X and y processing:
                    folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
                    sampleListName='matched86List' ################IMPORTANT!!! ###################
                    matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
                    sampleList= matched86List

                    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                          YName,targetDF,
                                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                          getTCRfeatures,sampleListName,sampleList,
                                          filterFeaturesByCorr,featureSelectionUsingModel,C,
                                          filterFeaturesByUnivariate,k)







### get results for each model:

In [ ]:
modelName_list=['LogisticRegression','LGBMClassifier','XGBClassifier','GaussianNB']


BestResultDF=pd.DataFrame()
for n, modelName in enumerate(modelName_list):
    print n, modelName
    folder='%s/TCR_real_data/Predictions/isCardio_predictions/%s/predictionDFs' %(MyPath,modelName)
    results=concat_summarizing_dfs_excel(folder)
    results=results.sort_values(by='roc_auc', ascending=False)
    f1='%s/TCR_real_data/Predictions/isCardio_predictions/%s/resultSummary_%s.xlsx' %(MyPath, modelName,cdate)
    results.to_excel(f1)
    BestResultDF.loc[n,'modelName']=modelName
    BestResultDF.loc[n,'best_roc_auc']=results['roc_auc'].max()
    BestResultDF.loc[n,'best_pr_auc_corrected']=results['pr_auc_corrected'].max()

BestResultDF

### compare results from different models:

In [ ]:
f1='%s/TCR_real_data/Predictions/isCardio_predictions/modelComparison_120718.xlsx' %MyPath
results=pd.read_excel(f1)
results

In [ ]:
results['roc_auc']=results['roc_auc'].astype(float)
fig,ax=plt.subplots()
colorList=['r','b','g','y']
x1=0
x2=0.4
for n,model in enumerate(results.Model.unique()):
    print n,model
    df=results[results.Model==model].drop('Model',axis=1).set_index('Features')
#     print df
    ROC=list(df.loc[:,'roc_auc'])
    print ROC
    PR=list(df.loc[:,'pr_auc_corrected'])
    print PR
    

#     arrow1=ax.arrow(x1+0.05*(n+1), ROC[0],0, ROC[1]-ROC[0],width=0.005,head_width=0.01, head_length=0.02, 
#              label=model,fc=colorList[n], ec=colorList[n])
#     arrow2=ax.arrow(x2+0.05*(n+1), PR[0],0, PR[1]-PR[0],width=0.005,head_width=0.01, head_length=0.02, 
#                     fc=colorList[n], ec=colorList[n])

    ax.axvline(x=x1+0.05*(n+1),ymin=ROC[0],ymax=ROC[1],lw=4,marker='^',ms=10,color=colorList[n],label=model)
    ax.axvline(x=x2+0.05*(n+1),ymin=PR[0],ymax=PR[1],lw=4,marker='^',ms=10,color=colorList[n])
#     x1list=[x1+0.05*n for i in range(2)]
#     print x1list
#     ax.plot(x=x1list,y=PR,color=colorList[n])
# #     ax.plot(x=[x2+0.05*n,x2+0.05*n],y=PR,color=colorList[n])
plt.xticks([0.1,0.5],['roc_auc','pr_auc_corrected'],fontsize=20)
plt.legend()
ax.grid(False)
ax.set_xlim(0,0.8)
# ax.set_xlabel('Metric')
ax.set_title('ROC-AUC and PR-AUC (corrected)\nfor model prediction using only phenotypes and phenotypes+TCRseqPCA+TCRfeatures')
plt.show()


# get sample list:
if necessary - generate matched cohorts and get the sample list

## generate new matched cohorts:

## ** get sample list

the function gen_sampleList_from_Folder was copied to SampleFileFunctions

In [ ]:
folder='%s/SamplesForAnalysis_corrected' %datasetFolder

matched86List=gen_sampleList_from_Folder(folder,'PNP530Cardio126Matched86')
print len(matched86List)
print matched86List[:5]

# ** get shared sequence matrix


## if necessary - generate new shared sequence matrix

## ** load shared sequence matrix

In [ ]:
TCRdfShortName=TCRdfName.replace('sharingMatrix_PNP530Cardio126MatchedSamples_','')
TCRdfShortName=TCRdfShortName.replace('__','_')

f1='%s/sharingAnalysis/%s' %(datasetFolder,TCRdfName)
TCRdf=pd.read_pickle(f1)
print TCRdf.shape
print TCRdf.iloc[:4,:4]

## **generate PCs as features:

In [ ]:
n_comp=5
isSparse=True

PCAdf=PCAfunc(TCRdf,n_comp,isSparse)
PCAdf=PCAdf.drop(['BDindex','isCardio'],axis=1)
PCAdf.head()

# ** get feature DF

##  generate new feature DF if necessary

go to notebook 'Generate feature DF' and generate the necessary dfs

### calculate features - from PopulationAnalaysis_new_version.ipynb

In [ ]:
data_folder=datasetFolder.replace('%s/' %MyPath,'')
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
for n,sample_name in enumerate(filenames): 
#     if n>3:
        print  n,sample_name
        gen_descriptive_stats(sample_name,data_folder,newColumnList=None)
        gen_geneUsageCount(sample_name,data_folder,newColumnList=None)

### generate feature DFs - based on 'Generate features DF' notebook

In [ ]:
seqTypeList = ['Total', 'Prod','nonProd']
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/Total' %data_folder
FeatureGroups=listdir(dfs_folder)
for seqType in seqTypeList:
    print seqType
    gen_featureSummaryDF_forSeqType(seqType,data_folder,datasetName,FeatureGroups)

In [ ]:
data_folder=datasetFolder.replace('%s/' %MyPath,'')
gen_merged_feature_df_for_dataset(data_folder,datasetName)

## ** load feature DF

In [ ]:
f2='%s/featureSummaryDFs/%s_allFeatures' %(datasetFolder,datasetName)
TCRfeatureDF=pd.read_pickle(f2)
print TCRfeatureDF.shape
print TCRfeatureDF.iloc[:4,:4]


## ** fillna's in featureDF

In [ ]:
#get number of columns with number of nan values?
print 'columns with top numbers of nan values:'
print TCRfeatureDF.isnull().sum().sort_values(ascending=False).head()

print 'value counts of nan numbers per column:'
print TCRfeatureDF.isnull().sum().value_counts(dropna=False).sort_values(ascending=False).head()

# find how many columns include nans:
print 'number of columns with nan values = %s' %len(TCRfeatureDF.loc[:,TCRfeatureDF.isnull().sum()>0].columns.tolist())
print 'total number of columns is %s' %len(TCRfeatureDF.columns)

#get list of columns that inlcude nans and should be fillna-ed with 0s:
columnsToFill=TCRfeatureDF.loc[:,TCRfeatureDF.isnull().sum()>0].columns.tolist()
columnsToFill_clean=[x for x in columnsToFill if 'norm' not in x]
print 'number of columns that include nans and their name doesnt include -norm- is %s' %len(columnsToFill_clean)
print 'these columns will be fillna-ed with 0s'

#fillna with zeros
TCRfeatureDF[columnsToFill_clean]=TCRfeatureDF[columnsToFill_clean].fillna(0)

#check how many columns remained with nans:
print 'now the number of columns with nan values is %s' %len(TCRfeatureDF.loc[:,TCRfeatureDF.isnull().sum()>0].columns.tolist())


# ** get full phenotype and target DF and filter if necessary

## generate phenotype df if necessary

### get the original phenotype file(s) - concat if necessary

In [ ]:
PNPphenotypeFile='%s/TCR_real_data/NewPhenotypicData/PNP530_AgeGenderBMIcreatSmoking.xlsx' %MyPath
CardioPhenotypeFile='%s/TCR_real_data/CardioSamples/phenotypicData/Cardio126phen.xlsx' %MyPath

PNPphenotypeDF=pd.read_excel(PNPphenotypeFile).set_index('BD')
CardiophenotypeDF=pd.read_excel(CardioPhenotypeFile).set_index('BD')

if datasetType=='PNP':
    phenotypeDF=PNPphenotypeDF
elif datasetType=='PNP_Cardio':
    phenotypeDF=pd.concat([PNPphenotypeDF,CardiophenotypeDF])
    phenotypeDF=phenotypeDF.drop(['RegistrationCode','DM','Diagnosis'],axis=1)


phenotypeDF['isCardio']=np.where(phenotypeDF.index.str.replace('BD','').astype(int)>949,1,0)

print phenotypeDF.shape
print phenotypeDF.tail()

### generate dummy variables:

In [ ]:
#the function gen_dummies was copied to Feature_phenotype_functions.py

In [ ]:
df=phenotypeDF
toDummyColList=['Gender','Smoking','PCRplate']

phenotypeDF_new=gen_dummies(phenotypeDF,toDummyColList)
phenotypeDF_new.head()

In [ ]:
f2='%s/TCR_real_data/PNP530Cardio126Combined/Phenotypes/PNP530Cardio126_phen_withDummies.xls' %MyPath
phenotypeDF_new.to_excel(f2)

## ** get phenotype df:

In [ ]:
phenotypeDF=pd.read_excel(phenotypefile).set_index('BD')
print phenotypeDF.shape
phenotypeDF.tail()

In [ ]:
# Xtypes:
allNum=['Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Gender_Male', 'Smoking_Past', 'Smoking_Yes',
       'PCR_Plate1', 'PCR_Plate10', 'PCR_Plate2', 'PCR_Plate3',
       'PCR_Plate4', 'PCR_Plate5', 'PCR_Plate6', 'PCR_Plate7',
       'PCR_Plate8', 'PCR_Plate9']
small=[ 'Age', 'BMI', 'Creatinine', 'isCardio', 'Gender_Male','Smoking_Past', 'Smoking_Yes']
smallNoCardio=['Age', 'BMI', 'Creatinine', 'Gender_Male','Smoking_Past', 'Smoking_Yes']
allNumNoPlate=['Age', 'BMI', 'Creatinine', 'isCardio', 'nTemplates', 'Gender_Male', 'Smoking_Past', 'Smoking_Yes']

if phenotypeDFname=='small':
    Xcols=small
elif phenotypeDFname=='allNum':
    Xcols=allNum
elif phenotypeDFname=='smallNoCardio':
    Xcols=smallNoCardio
elif phenotypeDFname=='allNumNoPlate':
    Xcols=allNumNoPlate

In [ ]:
phenotypeDF=phenotypeDF[Xcols]
print phenotypeDF.shape

## **get targetDF

In [ ]:
y=targetDF[targetName]
print y.shape
print y

# feature selection

# **prediction

## **define which features to use in the model:

In [ ]:
#build X from its components:

XcomponentList=[]
if usePhenotype:
    XcomponentList.append(phenotypeDF)
if useTCRdf:
    XcomponentList.append(TCRdf)
if useTCRfeatures:
    XcomponentList.append(TCRfeatureDF)
if usPCAdf:
    XcomponentList.append(PCAdf)
print 'number of Xcomponents to be used is %s' %len(XcomponentList)

## ** process X:

In [ ]:
#edit sample names and merge all X components:
for n,comp in enumerate(XcomponentList):
    print n
    print 'comp shape is %s_%s' %(comp.shape[0],comp.shape[1])
    print 'editing sample names..'
    comp=editSampleNames(comp)
    if n==0:
        X=comp
    else:
        X=pd.merge(X,comp, how='inner', right_index=True,left_index=True)
print 'combined X shape is %s_%s' %(X.shape[0],X.shape[1])
print len(X.index.tolist())

In [ ]:
#if model is 'logit', drop columns that contains nans, except for phenotype columns; fillna in phenotype columns
#and check for rows with nan values:

if modelName=='logit':
    nullSum=X.isnull().sum()
    ignoreColumns=nullSum[nullSum>0].index.tolist()
    for column in phenotypeDF.columns.values:
        if column in ignoreColumns:
            ignoreColumns.remove(column)
    X=X.drop(ignoreColumns,axis=1)
    print 'X shape after dropping nan columns is %s_%s' %(X.shape[0],X.shape[1])
    
    for column in phenotypeDF.columns.values:
        X[column]= X[column].fillna(X[column].median())
        
    X=X.dropna(how='any')
    print 'Xshape after dropping rows with nans=%s_%s' %(X.shape[0],X.shape[1])


In [ ]:
# drop samples that are not in y

ysamples=y.index.tolist()
X=X.loc[ysamples,:]
X=X.dropna(how='all')
print X.shape


## enable feature selection

## process Y:


In [ ]:
modelName='logit'

In [ ]:
if modelName=='logit':
    thresholdFracNans=0.01
elif modelName=='LGBMClassifier':
    thresholdFracNans=0.4

thresholdNans=len(X)*thresholdFracNans
nullSum=X.isnull().sum()

ignoreColumns=nullSum[nullSum>thresholdNans].index.tolist()
for column in phenotypeDF.columns.values:
#     print len(ignoreColumns)
#     print column
    if column in ignoreColumns:
        ignoreColumns.remove(column)
# print ignoreColumns
print 'number of columns with nan rates larger then %s = %s' %(thresholdFracNans,len(ignoreColumns))
X=X.drop(ignoreColumns,axis=1)
print 'Xshape after dropping high nan rate columns=%s_%s' %(X.shape[0],X.shape[1])
if modelName=='logit':
    X=X.dropna(how='any')
    print 'Xshape after dropping rows with nans=%s_%s' %(X.shape[0],X.shape[1])

* if model type = 'logit' - drop columns with more than 15% missing
                        
* if model type='xgboost' - drop columns with more than 25% missing

In [ ]:
# leave only rows that appear in X
print 'y shape before filtering for X sample is %s' %y.shape[0]
Xsamples=X.index.tolist()
y=y.loc[Xsamples]
# y=y.dropna(how='all')
print 'y shape after filtering for X sample is %s' %y.shape[0]

print y.head()
print y.tail()
    

## prediction function 
including CV design, evaluation metrics, plots

## run both models for the matched samples using all features/drop TCR sequences/drop also TCR features

In [ ]:
def predictBinaryByDistMat(Y,YName,X,XName,ResultFolder,model,model_name,model_params,n_splits,useCV=True,stratifiedCV=True):
    
    #(1) arrange result folders:
    print 'arranging  result folders:'
    predResultsDF=pd.DataFrame()
    
    predResultsDFFolder='%s/%s_predictions/predictionDFs' %(ResultFolder,YName) #define folder for all result dfs in this model
    if not isdir(predResultsDFFolder):
        makedirs(predResultsDFFolder)
        print 'generating predResultsDFFolder %s' %predResultsDFFolder
        
    predResultsfigFolder='%s/%s_predictions/figs' %(ResultFolder,YName) #define folder for figs in this model
    if not isdir(predResultsfigFolder):
        makedirs(predResultsfigFolder)
        print 'predResultsfigFolder %s' %predResultsfigFolder

    #generate result DF name based on model_params:
    d=OrderedDict(sorted(model_params.items(), key=lambda t: t[0]))
    try:
        d2=d.copy()
        del d2['numthreads']
    except:
        d2=d.copy()
    predResultsDFName='_'.join(['%s%s' %(key.replace('_',''), value) for (key, value) in d2.items()]) #generate a file name based on params
    predResultsDFName=predResultsDFName.replace('.','-')
    if useCV:
        predResultsDFName='%s_CV%s' %(predResultsDFName,n_splits)
    
    predResultsDFName='%s_%s' %(XName,predResultsDFName)
    
    predResultsDFfile='%s/%s' %(predResultsDFFolder,predResultsDFName)
    existingDFs=[f for f in listdir( predResultsDFFolder) if isfile(join(predResultsDFFolder, f))]
    

    if predResultsDFName not in existingDFs:  
        

        # common processing of X and y:
        # leave only common samples in each df (X and seq1data)
        X=X.loc[[str(x) for x in X.index.tolist() if x in Y.index],:]
        X=X.sort_index()
        print 'X shape is %s_%s' %(X.shape[0], X.shape[1])
        print 'the 100th sample in X is %s' %X.index[100]
        print X.iloc[:3,:3]
        Y=Y.loc[[str(x) for x in Y.index.tolist() if x in X.index]]
        Y=Y.sort_index()
#         oldColName=str(Y.columns.values[0])
#         Y=Y.rename(columns={oldColName:'Class'})
        print 'Y shape is %s' %(Y.shape[0])
        print 'the 100th sample in Y is %s' %Y.index[100]
        print Y.head(3)

        #(2) model fitting and predictions:

        if useCV:
            print 'splitting train_test using cross validation with %s splits...' %n_splits
            
            if stratifiedCV:
                group_kfold = StratifiedKFold(n_splits=n_splits)
                groups=None
            else:
                group_kfold = GroupKFold(n_splits=n_splits)
                groups = np.array(range(X.shape[0]))
                
        
            y_pred_df = pd.DataFrame(index=Y.index, columns=['pred_proba'])
            i=0
            for train_index, val_index in group_kfold.split(X, Y,groups):
                print i
                i+=1
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = Y.loc[X_train.index], Y.loc[X_val.index]
                
                print 'fraction of 1s in train set=%s' %(float(y_train.sum())/len(y_train))
                print 'fraction of 1s in test set=%s' %(float(y_val.sum())/len(y_val))

                # creating the model object
                m = model(**model_params)

                # fitting the training
                m.fit(X_train, y_train,
                        eval_set=[(X_val, y_val)],
                        early_stopping_rounds=None, verbose=-1)
                # getting the predictions for the test
                y_pred_proba = m.predict_proba(X_val)
                y_pred_df.loc[y_val.index, :] = np.expand_dims(y_pred_proba[:,1], 1)
        else:
            test_size = 0.33
            print 'using normal train_test split with test_size=%s' %test_size
            from sklearn.model_selection import train_test_split  
            # seed = 7
            
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, stratify=Y)
            print 'fitting model...'
            model = model
            model.fit(X_train, y_train)
            # make predictions for test data
            print 'predicting...'
            y_pred = model.predict(X_test)
            predictions = [round(value) for value in y_pred]
            # evaluate predictions
            from sklearn.metrics import accuracy_score
            accuracy = accuracy_score(y_test, y_pred)
            print("Accuracy: %.2f%%" % (accuracy * 100.0))
            y_pred_df = pd.DataFrame(index=Y.index, data={'pred':y_pred})

        #(4) plots:
        print 'generating plots...'

        # this plot shows the probabilities returned by the predictor colored by the class
        plt.figure(figsize=(3,2))
        plt.scatter(range(y_pred_df.shape[0]), y_pred_df.pred_proba, c=Y)
        plt.show()

        # plot ROC and PR curves

        from sklearn import metrics
        fpr, tpr, thresholds = metrics.roc_curve(Y+1, y_pred_df.pred_proba, pos_label=2)
        roc_auc = metrics.auc(fpr, tpr)
        print roc_auc

        plt.figure(figsize=(20,7))
        plt.subplot(1,2,1)
        lw = 2
        plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.3f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate', fontsize=15)
        plt.ylabel('True Positive Rate', fontsize=15)
        plt.title('ROC curve - ' , fontsize=20)
        plt.legend(loc="lower right", fontsize=15)

        plt.subplot(1,2,2)
        precision, recall, _ = metrics.precision_recall_curve(Y, y_pred_df.pred_proba)
        pr_auc = metrics.auc(recall, precision)

        plt.step(recall, precision, color='b', alpha=0.2, where='post')
        plt.fill_between(recall, precision, step='post', alpha=0.5,
                         color='darkorange', label='Precision Recall curve - AUC = {0:0.3f}'.format(pr_auc))
        plt.plot([0, 1], [Y.sum()/Y.shape[0], Y.sum()/Y.shape[0]], color='navy', lw=lw, linestyle='--')
        plt.xlabel('Recall', fontsize=15)
        plt.ylabel('Precision', fontsize=15)
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.title('Precision Recall curve', fontsize=20)
        plt.legend(loc="upper right", fontsize=15)

        if roc_auc>0.55:
            predResultsFigfile='%s/%s_ROC-PR.png' %(predResultsfigFolder,predResultsDFName)
            plt.savefig(predResultsFigfile, bbox_inches='tight')

        plt.show()

        #(5) generate summarizing df:
        print 'generating summarizing df'
        predResultsDF.loc[0,'Yname']=YName
        predResultsDF.loc[0,'Xname']=XName
        predResultsDF.loc[0,'model_name']=model_name
        predResultsDF.loc[0,'roc_auc']=round(roc_auc,3)
        predResultsDF.loc[0,'pr_auc']=round(pr_auc,3)
        predResultsDF.loc[0,'useCV']=useCV
        if useCV:
            predResultsDF.loc[0,'n_splits']=n_splits
        
        
        for (key, value) in OrderedDict(sorted(model_params.items(), key=lambda t: t[0])).items():
            predResultsDF.loc[0,key]=value
        
        nPos=int(Y.sum())
        nNeg=len(Y)-nPos
        
        print nPos, nNeg

        predResultsDF.loc[0,'nPos']=nPos
        predResultsDF.loc[0,'nNeg']=nNeg
        
        
        predResultsDF.to_pickle(predResultsDFfile)
        print predResultsDFfile
        
    else:
        print 'this prediction already exists'
    
    return predResultsDF
    
    
    
    
    
    
    

In [ ]:
Y=y
YName='isCardio'
X=X
XName='PNP530Cardio126Matchedss9000_onlyPhenotypes'
ResultFolder='%s/TCR_real_data/isCardioPredictions' %MyPath
if not isdir(ResultFolder):
    makedirs(ResultFolder)
model= lgb.LGBMClassifier
model_name='LGBMClassifier'
n_splits=3
useCV=True
stratifiedCV=True

learning_rate=0.1
n_estimators=100
max_depth=3
model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}



predResultsDF=predictBinaryByDistMat(Y,YName,X,XName,ResultFolder,model,model_name,model_params,n_splits,useCV=True,stratifiedCV=True)
    

In [ ]:
predResultsDF

## run both models for the All samples (not only matched) using all features/drop TCR sequences/drop also TCR features

## add phenotypes? (CRP, blood pressure)